# ETL Pipeline Preparation
### 1. Import libraries and load datasets

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine

In [ ]:
messages = pd.read_csv('messages.csv')
messages.head()

In [ ]:
categories = pd.read_csv('categories.csv')
categories.head()

### 2. Merge datasets

In [ ]:
df = messages.merge(categories, how='outer', on='id')
df.head()

### 3. Split categories into separate columns

In [ ]:
categories = df.categories.str.split(';', expand=True)
categories.head()

In [ ]:
# Extract column names from the first row (strip trailing "-0"/"-1")
row = categories.iloc[0, :]
category_colnames = row.apply(lambda x: x[:-2])
print(category_colnames)

In [ ]:
categories.columns = category_colnames
categories.head()

### 4. Convert category values to binary (0 or 1)

In [ ]:
for column in categories:
    categories[column] = categories[column].str[-1]
    categories[column] = categories[column].astype(int)
categories.head()

### 5. Replace the original categories column with the new binary columns

In [ ]:
df = df.drop('categories', axis=1)
df.head()

In [ ]:
df = pd.concat([df, categories], axis=1)
df.head()

### 6. Remove duplicates

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()

In [ ]:
# Confirm duplicates removed
df.duplicated().sum()

### 7. Save to SQLite database

In [ ]:
engine = create_engine('sqlite:///InsertDatabaseName.db')
df.to_sql('ETL', engine, index=False)